In [31]:
import numpy as np
from sklearn import linear_model
import pandas as pd
from sklearn import preprocessing

In [32]:
clf = linear_model.SGDClassifier()

In [33]:
save_dir='../primary_data/'

In [70]:
dems = pd.read_csv('/home/jasonmellone/Documents/nyu/DSQF/stitched_data/county_dems.csv')

In [71]:
reps = pd.read_csv('/home/jasonmellone/Documents/nyu/DSQF/stitched_data/county_reps.csv')

In [72]:
le = preprocessing.LabelEncoder()

In [73]:
le.fit(list(set(dems['1st'].unique()).union(dems['2nd'].unique()).union(dems['3rd'].unique())))

LabelEncoder()

In [74]:
dems['1st_encoded'] = le.transform(dems['1st'].values)

In [75]:
dems['2nd_encoded'] = le.transform(dems['2nd'].values)

In [76]:
dems['3rd_encoded']= le.transform(dems['3rd'].values)